## Section 1: Setup & Environment Configuration

In [15]:
import os
from pathlib import Path
from dotenv import dotenv_values

# 找到 .env 檔案
env_file = Path.cwd().parent.parent / ".env"
if not env_file.exists():
    env_file = Path.home() / "Documents/workspace/CasualTrader/backend/.env"

# 讀取並設定環境變數
if env_file.exists():
    env_vars = dotenv_values(env_file)
    for key, value in env_vars.items():
        os.environ[key] = value.strip('"').strip("'") if value else ""

# 取得 API Key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GITHUB_PERSONAL_ACCESS_TOKEN = os.getenv("GITHUB_PERSONAL_ACCESS_TOKEN")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

print(f"✓ .env 已載入: {env_file}")
print(f"✓ OPENAI_API_KEY: {OPENAI_API_KEY[:20]}...{OPENAI_API_KEY[-20:]}" if OPENAI_API_KEY else "✗ 未設定")
print(f"✓ GITHUB_PERSONAL_ACCESS_TOKEN: {GITHUB_PERSONAL_ACCESS_TOKEN[:20]}...{GITHUB_PERSONAL_ACCESS_TOKEN[-20:]}" if GITHUB_PERSONAL_ACCESS_TOKEN else "✗ 未設定")
print(f"✓ TAVILY_API_KEY: {TAVILY_API_KEY[:20]}...{TAVILY_API_KEY[-20:]}" if TAVILY_API_KEY else "✗ 未設定")

✓ .env 已載入: /Users/sacahan/Documents/workspace/CasualTrader/backend/.env
✓ OPENAI_API_KEY: sk-proj-T_8ONKeh5L1O...l90oicgENtP8nRj3ljgA
✓ GITHUB_PERSONAL_ACCESS_TOKEN: ghp_HJYjAlbBoxrUSnE0...9nt7LU6xZnoIjQ2gt793
✓ TAVILY_API_KEY: tvly-dev-ePjxQERPzxI...Jr3TphaFNjo6Km737Hqc


## Section 2: LiteLLM GitHub Copilot - Authentication & Basic Usage

In [ ]:
from litellm import completion, acompletion
import asyncio

# Test 1: Non-streaming completion with GitHub Copilot
# Note: On first run, LiteLLM will prompt with OAuth device code

def test_github_copilot_basic():
    """
    Test basic GitHub Copilot completion via LiteLLM.
    
    First execution triggers OAuth device flow:
    - LiteLLM displays a device code and verification URL
    - Visit the URL and enter the code
    - Authenticate with your GitHub account
    - OAuth tokens are cached for future use
    """

    model = "github_copilot/gemini-2.5-pro"

    print("\n=== Test 1: Basic Completion (Non-Streaming) ===")
    print(f"Model: {model}")
    print("Status: Awaiting response...\n")
    
    try:
        response = completion(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": "Write a Python function to calculate the Fibonacci sequence up to n numbers, just python code without any explanation",
                }
            ],
            # temperature=0.7,
            max_tokens=500,
            extra_headers={
                "editor-version": "vscode/1.85.1",
                "Copilot-Integration-Id": "vscode-chat",
            },
        )
        
        # print("Response received:")
        print("-" * 50)
        print(response["choices"][0]["message"]["content"])
        print("-" * 50)
        # print(f"\nUsage: {response.get('usage', 'N/A')}")
        
    except Exception as e:
        print(f"Error: {type(e).__name__}")
        print(f"Message: {str(e)}")
        print("\nNote: This is expected if:")
        print("  1. OAuth authentication is needed (first run)")
        print("  2. GitHub Copilot subscription is not active")
        print("  3. Network connectivity issue")

# Run the test
test_github_copilot_basic()

## Section 3: LiteLLM GitHub Copilot - Streaming

In [ ]:
from litellm import completion, acompletion


def test_github_copilot_streaming():
    """
    Test streaming responses from GitHub Copilot via LiteLLM.
    
    Streaming is useful for real-time feedback in UI applications.
    """

    model = "github_copilot/gpt-5-mini"

    print("\n=== Test 2: Streaming Completion ===")
    print(f"Model: {model}")
    print("Status: Streaming response...\n")
    
    try:
        stream = completion(
            model=model,
            messages=[
                {
                    "role": "user",
                    "content": "Explain async/await in Python with a simple example"
                }
            ],
            stream=True,
            # temperature=0.8,
            max_tokens=300,
            extra_headers={
                "editor-version": "vscode/1.85.1",
                "Copilot-Integration-Id": "vscode-chat"
            }
        )
        
        print("Streaming response:")
        print("-" * 50)
        
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        
        print("\n" + "-" * 50)
        print(f"\nTotal characters received: {len(full_response)}")
        
    except Exception as e:
        print(f"Error: {type(e).__name__}: {str(e)}")

# Run the test
test_github_copilot_streaming()

## Section 4: Example - Using GitHub Copilot with Agents SDK

In [ ]:
# Example configuration for using GitHub Copilot with OpenAI Agents

from agents import Agent, Runner, function_tool, ModelSettings, gen_trace_id, trace
from agents.extensions.models.litellm_model import LitellmModel
import json


model = "github_copilot/gpt-5-mini"
print(f"\nUsing model: {model}")

model_settings = ModelSettings(
    include_usage=True,
    extra_headers={
        "editor-version": "vscode/1.85.1",
        "Copilot-Integration-Id": "vscode-chat",
    }
)


@function_tool
def get_weather(city: str):
    print(f"[debug] getting weather for {city}")
    return f"The weather in {city} is rainy."

agent = Agent(
    name="Assistant",
    instructions="You only respond in haikus.",
    # model=LitellmModel(model=model, api_key=GITHUB_PERSONAL_ACCESS_TOKEN),
    model=LitellmModel(model=model),
    tools=[get_weather],
    model_settings=model_settings,
)

trace_id = gen_trace_id()
with trace(workflow_name="lab_agent", trace_id=trace_id):
    result = await Runner.run(agent, "What's the weather in Tokyo?")
    print(json.dumps(result.final_output, indent=2))

print("\n=== GitHub Copilot + Agents Configuration ===")

## Section 5: Agent with MCP Tool Integration

In [16]:
from contextlib import AsyncExitStack
from agents import Agent, Runner, function_tool, ModelSettings, gen_trace_id, trace
from agents.extensions.models.litellm_model import LitellmModel
from agents.mcp import MCPServerStdio
import json
from datetime import datetime
import litellm

litellm._turn_on_debug()
model = "github_copilot/claude-haiku-4.5"
print(f"\nUsing model: {model}")

# Initialize MCP servers
async def get_tavily_mcp():
    """
    初始化 MCP 伺服器並註冊到 exit stack

    Raises:
        Exception: 初始化失敗
    """
    global _exit_stack  # 修正 UnboundLocalError
    try:
        if not _exit_stack:
            _exit_stack = AsyncExitStack() 

        # Tavily MCP Server
        # 參數在 MCPServerStdio 的 params 內配置
        tavily_mcp = await _exit_stack.enter_async_context(
            MCPServerStdio(
                name="tavily_mcp",
                params={
                    "command": "npx",
                    "args": ["-y", "tavily-mcp@latest"],
                    "env": {"TAVILY_API_KEY": f"{TAVILY_API_KEY}"},
                },
                client_session_timeout_seconds=30.0,
            )
        )
        print("tavily_mcp server initialized")
        return tavily_mcp

    except Exception as e:
        print(f"Failed to initialize MCP server: {e}")
        # 如果初始化失敗，清理已創建的資源
        if _exit_stack:
            await _exit_stack.aclose()
            _exit_stack = None

tavily_mcp = await get_tavily_mcp()

agent = Agent(
    name="Analyst",
    instructions="You using web search to answer user queries.",
    model=LitellmModel(model=model),
    mcp_servers=[tavily_mcp],
    model_settings=ModelSettings(
        include_usage=True,
        tool_choice="required",
        extra_headers={
            "editor-version": "vscode/1.85.1",
            "Copilot-Integration-Id": "vscode-chat",
        },
    ),
)

trace_id = gen_trace_id()
with trace(workflow_name="lab_section_5", trace_id=trace_id):
    message = f"明天東京的天氣如何？ (目前時間：{datetime.now().strftime('%Y-%m-%d %H:%M')})"
    print(f"message: {message}")
    print("-" * 50)
    result = await Runner.run(agent, message)
    print(json.dumps(result.final_output, indent=2, ensure_ascii=False))

print("\n=== Agent with MCP Tool Integration ===")


Using model: github_copilot/claude-haiku-4.5


19:04:31 - LiteLLM:DEBUG: utils.py:366 - 

19:04:31 - LiteLLM:DEBUG: utils.py:366 - Request to litellm:
19:04:31 - LiteLLM:DEBUG: utils.py:366 - litellm.acompletion(model='github_copilot/claude-haiku-4.5', messages=[{'content': 'You using web search to answer user queries.', 'role': 'system'}, {'role': 'user', 'content': '明天東京的天氣如何？ (目前時間：2025-10-30 19:04)'}], tools=[{'type': 'function', 'function': {'name': 'tavily-search', 'description': "A powerful web search tool that provides comprehensive, real-time results using Tavily's AI search engine. Returns relevant web content with customizable parameters for result count, content type, and domain filtering. Ideal for gathering current information, news, and detailed web content analysis.", 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search query'}, 'search_depth': {'type': 'string', 'enum': ['basic', 'advanced'], 'description': "The depth of the search. It can be 'basic' or 'advanced'", 'de

tavily_mcp server initialized
message: 明天東京的天氣如何？ (目前時間：2025-10-30 19:04)
--------------------------------------------------


19:04:32 - LiteLLM:DEBUG: utils.py:2111 - Model not found or error in checking supports_reasoning support. You passed model=claude-haiku-4.5, custom_llm_provider=None. Error: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call. You passed model=claude-haiku-4.5
 Pass model as E.g. For 'Huggingface' inference endpoints pass in `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers
19:04:32 - LiteLLM:INFO: utils.py:3388 - 
LiteLLM completion() model= claude-haiku-4.5; provider = github_copilot
19:04:32 - LiteLLM:DEBUG: utils.py:3391 - 
LiteLLM: Params passed to completion() {'model': 'claude-haiku-4.5', 'functions': None, 'function_call': None, 'temperature': None, 'top_p': None, 'n': None, 'stream': False, 'stream_options': None, 'stop': None, 'max_tokens': None, 'max_completion_tokens': None, 'modalities': None, 'prediction': None, 'audio': None, 'presence_penalty': None, 'frequency_penalty': Non


Provider List: https://docs.litellm.ai/docs/providers



19:04:34 - LiteLLM:DEBUG: litellm_logging.py:1027 - RAW RESPONSE:
{"id": "msg_01A6erRR76BoAsAvCKzA5bGK", "choices": [{"finish_reason": "tool_calls", "index": null, "logprobs": null, "message": {"content": null, "refusal": null, "role": "assistant", "annotations": null, "audio": null, "function_call": null, "tool_calls": [{"id": "toolu_01VDw7Eb9c1fstKVyyLk9abY", "function": {"arguments": "{\"query\":\"\u660e\u5929\u6771\u4eac\u5929\u6c23 2025\u5e7410\u670831\u65e5\",\"country\":\"japan\",\"max_results\":10}", "name": "tavily-search"}, "type": "function"}]}}], "created": 1761822274, "model": "claude-haiku-4.5", "object": null, "service_tier": null, "system_fingerprint": null, "usage": {"completion_tokens": 89, "prompt_tokens": 3264, "total_tokens": 3353, "completion_tokens_details": null, "prompt_tokens_details": {"audio_tokens": null, "cached_tokens": 0}}}


19:04:34 - LiteLLM:DEBUG: litellm_logging.py:2918 - Filtered callbacks: []
19:04:34 - LiteLLM:DEBUG: cost_calculator.py:775 - sele


Provider List: https://docs.litellm.ai/docs/providers



19:04:41 - LiteLLM:DEBUG: litellm_logging.py:1027 - RAW RESPONSE:
{"id": "msg_01HxNnxob64ojVAzdoKdU4ad", "choices": [{"finish_reason": "stop", "index": null, "logprobs": null, "message": {"content": "\u611f\u8b1d\u60a8\u7684\u63d0\u554f\uff01\u6839\u64da\u641c\u7d22\u7d50\u679c\uff0c\u6211\u9700\u8981\u544a\u77e5\u60a8\u4e00\u500b\u91cd\u8981\u7684\u60c5\u6cc1\uff1a\n\n## \u6642\u9593\u554f\u984c\n\n\u60a8\u63d0\u5230\u7684\u7576\u524d\u6642\u9593\u662f **2025-10-30 19:04**\uff0c\u9019\u610f\u5473\u8457\u660e\u5929\u662f **2025\u5e7410\u670831\u65e5**\u3002\u4f46\u6839\u64da\u641c\u7d22\u7d50\u679c\u4e2d\u7684\u65e5\u672c\u6c23\u8c61\u6578\u64da\uff0c\u9019\u4e9b\u6578\u64da\u662f\u904e\u53bb\u5df2\u767c\u751f\u7684\u5929\u6c23\u8a18\u9304\uff0810\u6708\u5df2\u7d93\u904e\u5b8c\uff09\uff0c\u800c\u6211\u627e\u5230\u7684\u9810\u5831\u4fe1\u606f\u4e0d\u5920\u660e\u78ba\u3002\n\n## \u53ef\u7528\u7684\u4fe1\u606f\n\n\u6839\u64da\u641c\u7d22\u7d50\u679c\uff0c\u6211\u80fd\u770b\u5230\uff1a\n- 

"感謝您的提問！根據搜索結果，我需要告知您一個重要的情況：\n\n## 時間問題\n\n您提到的當前時間是 **2025-10-30 19:04**，這意味著明天是 **2025年10月31日**。但根據搜索結果中的日本氣象數據，這些數據是過去已發生的天氣記錄（10月已經過完），而我找到的預報信息不夠明確。\n\n## 可用的信息\n\n根據搜索結果，我能看到：\n- 搜索結果中提到了「10月31日」的天氣信息，但詳細內容未完整顯示\n- 一條結果提到「明後日31日(金)は太平洋側で雨雲発達」（後天31日金曜日，太平洋側會有雨雲發展）\n\n## 建議\n\n由於搜索結果的信息有限，我建議您直接查閱以下日本官方天氣預報來源：\n\n1. **日本氣象廳** (JMA): https://www.jma.go.jp/ \n2. **天氣新聞** (Weather News): https://weathernews.jp/\n3. **Tenki.jp**: https://tenki.jp/\n4. **AccuWeather 日本版**: https://www.accuweather.com/ja/jp/\n\n這些網站會提供最準確的實時天氣預報。根據現在是晚上7點，您應該能找到明天31日的詳細預報。"

=== Agent with MCP Tool Integration ===
